Preprocess speech sound with background noises

In [6]:
import torch
import torchaudio
import torchaudio.functional as F

print(torch.__version__)
print(torchaudio.__version__)

import matplotlib.pyplot as plt

2.1.1+cpu
2.1.1+cpu


Download assets

In [7]:
from IPython.display import Audio

#from torchaudio.transforms import AudioEffectsChain
from torchaudio.utils import download_asset

SAMPLE_WAV = download_asset("tutorial-assets/steam-train-whistle-daniel_simon.wav")
SAMPLE_RIR = download_asset("tutorial-assets/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo-8000hz.wav")
SAMPLE_SPEECH = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042-8000hz.wav")
SAMPLE_NOISE = download_asset("tutorial-assets/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo-8000hz.wav")

In [8]:
# Load the data
#SAMPLE_WAV = "C:/Users/benny/Desktop/Y4S1/Deep_Speech_Technology/Project/data/sample/steam-train-whistle-daniel_simon.wav"
waveform1, sample_rate = torchaudio.load(SAMPLE_WAV, channels_first=False)

# Define effects
effect = ",".join(
    [
        "lowpass=frequency=300:poles=1",  # apply single-pole lowpass filter
        "atempo=0.8",  # reduce the speed
        "aecho=in_gain=0.8:out_gain=0.9:delays=200:decays=0.3|delays=400:decays=0.3"
        # Applying echo gives some dramatic feeling
    ],
)

# Apply effects
def apply_effect(waveform, sample_rate, effect):
    #effector = AudioEffectsChain(effects=[effect])
    effector = torchaudio.io.AudioEffector(effect=effect)
    return effector.apply(waveform, sample_rate)


waveform2 = apply_effect(waveform1, sample_rate, effect)

print(waveform1.shape, sample_rate)
print(waveform2.shape, sample_rate)


Failed to load FFmpeg 6 extension.
Traceback (most recent call last):
  File "c:\Users\benny\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\_extension\utils.py", line 128, in _find_ffmpeg_extension
    return _find_versionsed_ffmpeg_extension(ffmpeg_ver)
  File "c:\Users\benny\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\_extension\utils.py", line 113, in _find_versionsed_ffmpeg_extension
    _try_access_avutil(ffmpeg_ver)
  File "c:\Users\benny\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\_extension\utils.py", line 89, in _try_access_avutil
    torchaudio.lib._torchaudio.find_avutil(libavutil)
RuntimeError: error in LoadLibrary for avutil-58.dll. WinError 126: The specified module could not be found.

Failed to load FFmpeg 5 extension.
Traceback (most recent call last):
  File "c:\Users\benny\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\_extension\utils.py", line 128, in _find_ffmpeg_extensio

RuntimeError: StreamWriter requires FFmpeg extension which is not available. Please refer to the stacktrace above for how to resolve this.

Add noise

In [ ]:
sample_rate = 16000
original_speech, sample_rate = torchaudio.load(SAMPLE_SPEECH)

#plot_specgram(original_speech, sample_rate, title="Original")

# Apply RIR
rir_applied = F.fftconvolve(speech, rir)

#plot_specgram(rir_applied, sample_rate, title="RIR Applied")

# Add background noise
# Because the noise is recorded in the actual environment, we consider that
# the noise contains the acoustic feature of the environment. Therefore, we add
# the noise after RIR application.
noise, _ = torchaudio.load(SAMPLE_NOISE)
noise = noise[:, : rir_applied.shape[1]]

snr_db = torch.tensor([8])
bg_added = F.add_noise(rir_applied, noise, snr_db)

#plot_specgram(bg_added, sample_rate, title="BG noise added")

# Apply filtering and change sample rate
effect = ",".join(
    [
        "lowpass=frequency=4000:poles=1",
        "compand=attacks=0.02:decays=0.05:points=-60/-60|-30/-10|-20/-8|-5/-8|-2/-8:gain=-8:volume=-7:delay=0.05",
    ]
)

filtered = apply_effect(bg_added.T, sample_rate, effect)
sample_rate2 = 8000

#plot_specgram(filtered.T, sample_rate2, title="Filtered")

# Apply telephony codec
#codec_applied = apply_codec(filtered, sample_rate2, "g722")
#plot_specgram(codec_applied.T, sample_rate2, title="G.722 Codec Applied")

Sample

In [ ]:
Audio(noise, rate=sample_rate)